# Error fields
This notebook illustrates the error field computation using different techniques, including the so-called *Clever Poor Man's Error* (CPME) method.

In [ ]:
using DIVAnd
using Makie
using CairoMakie
using GeoMakie
using Dates
using Statistics
using LinearAlgebra
include("../config.jl")

## Data reading
Download the file (it not already done) and read it.

In [ ]:
varname = "Salinity"
filename = salinityprovencalfile
download_check(salinityprovencalfile, salinityprovencalfileURL)

In [ ]:
obsval,obslon,obslat,obsdepth,obstime,obsid = loadobs(Float64, salinityprovencalfile, varname);

## Topography and grid definition
See the notebook on [bathymetry](../2-Preprocessing/06-topography.ipynb) for more explanations.

Define domain and resolution, create the domain.

In [ ]:
dx = dy = 0.125/2.
lonr = 2.5:dx:12.
latr = 42.3:dy:44.6

mask,(pm,pn),(xi,yi) = DIVAnd_rectdom(lonr,latr);

Download the bathymetry file and load it.

In [ ]:
bathname = gebco04file
download_check(gebco04file, gebco04fileURL)

In [ ]:
bx,by,b = load_bath(bathname,true,lonr,latr);

Create a land-sea mask based on the bathymetry.

In [ ]:
mask = falses(size(b,1),size(b,2))

for j = 1:size(b,2)
    for i = 1:size(b,1)
        mask[i,j] = b[i,j] >=1.0
    end
end

## Data selection for example

Cross validation, error calculations etc. assume independant data.      
Hence do not take high-resolution vertical profiles with all data but restrict yourself to specific small depth range.      
Here we limit outselves to August data at surface:

In [ ]:
sel = (obsdepth .< 1) .& (Dates.month.(obstime) .== 8)

obsval = obsval[sel]
obslon = obslon[sel]
obslat = obslat[sel]
obsdepth = obsdepth[sel]
obstime = obstime[sel]
obsid = obsid[sel];
@show (size(obsval))
checkobs((obslon,obslat,obsdepth,obstime),obsval,obsid)

### Analysis
Analysis parameters have been calibrated in the other notebook [13-processing-parameter-optimization example.ipynb](13-processing-parameter-optimization). 

<div class="alert alert-block alert-warning"> 
 ⚠ If the statistical parameters are incorrectly estimated, the error fields are meaningless and only provide an idea of data coverage.
</div>

The analysis parameters are:

In [ ]:
len = 0.3
epsilon2 = 1.4;

Analysis `fi` using mean data as background.    
Structure `s` is stored for later use in error calculation.

In [ ]:
fi, s = DIVAndrun(mask,(pm,pn),(xi,yi),(obslon,obslat),obsval.-mean(obsval),len,epsilon2);

Create a simple plot of the analysis

In [ ]:
fig = Figure(size = (600, 350))
ga = GeoAxis(fig[1, 1];  dest = "+proj=merc", title = "Analysis and residuals", 
xticks=2.:1.:14, yticks=42.:1.:45.)
heatmap!(ga, bx, by, mask, colormap=Reverse(:binary))
hm = heatmap!(ga, lonr, latr, fi .+ mean(obsval), interpolate = false, colorrange=[37., 38.5])
# sc = scatter!(obslon, obslat, color=obsval, markersize=7, colorrange=[37., 38.5])
xlims!(ga, (bx[1], bx[end]))
ylims!(ga, (by[1], by[end]))
Colorbar(fig[2, 1], hm, vertical = false, label = "S")
fig

## Exact error and approximations

<div class="alert alert-block alert-info"> 
<b>NOTE</b> Details can be found in the publication mentioned at the beginning of the notebook.
</div>

First we define a utility function for plotting.

In [ ]:
function ploterr(exerr; vmin=0, vmax=1.5, cmap=Reverse(:hot), figtitle="Error field")
    fig = Figure(size = (600, 350))
    ga = GeoAxis(fig[1, 1];  dest = "+proj=merc", title = figtitle, 
    xticks=2.:1.:14, yticks=42.:1.:45.)
    heatmap!(ga, bx, by, mask, colormap=Reverse(:binary))
    hm = heatmap!(ga, lonr, latr, exerr, colormap=cmap, interpolate = false, colorrange=[vmin, vmax])
    # sc = scatter!(obslon, obslat, color=obsval, markersize=7, colorrange=[37., 38.5])
    xlims!(ga, (bx[1], bx[end]))
    ylims!(ga, (by[1], by[end]))
    Colorbar(fig[2, 1], hm, vertical = false)
    fig
end

### Exact error by hand

In the 2D case you can try to calculate the exact error expression. This demands the computationally expensive evaluation of `diag(s.P)` accessible via the analysis returned structure `s`. The latter is only available with `DIVAndrun`.

In [ ]:
exerr, = statevector_unpack(s.sv,diag(s.P),NaN)
ploterr(exerr, figtitle= "Error using P, scaled by global background variance")

Relative error by scaling with background variance `Berr` estimated using data with high errors

In [ ]:
epsilon2huge=1E6
fib, sb = DIVAndrun(mask,(pm,pn),(xi,yi),(obslon,obslat),obsval,len,epsilon2huge);
Berr,= statevector_unpack(sb.sv,diag(sb.P));

ploterr(exerr./Berr, figtitle="Scaled error")

In [ ]:
cpme = DIVAnd_cpme(mask,(pm,pn),(xi,yi),(obslon,obslat),obsval,len,epsilon2)

ploterr(cpme, figtitle="Clever poor man's error")

Do you see any difference between the exact and clever poor man's error ? 

## Difference between error fields

In [ ]:
ploterr(cpme-exerr./Berr, vmin=-0.2, vmax=0.2, cmap=Reverse(:RdYlBu), figtitle="Error on error")

Another approximation to the error field: AEXERR

In [ ]:
myerr,bjmb,fa,sa = DIVAnd_aexerr(mask,(pm,pn),(xi,yi),(obslon,obslat),obsval,len,epsilon2)
if myerr==0
    @error("No need to approximate error, use direct calculation")
else
    ploterr(myerr, figtitle="Unscaled almost exact error")
end

Scaled AXERR error

In [ ]:
if myerr==0
    @error("No need to approximate error, use direct calculation")
else
    ploterr(myerr./bjmb, figtitle="Scaled almost exact error")
end

# Exercise 
1. Modify the (L, $\epsilon^2$) parameters.
2. Re-run the analysis.
3. See how the error field behaves.

# Conclusion
In view of the uncertainties on statistical parameters (L, $\epsilon^2$), the *clever poor man's error* is generally a sufficient approximation for the error fields.     
This is the one implemented in the `DIVAndgo` high-level analysis function. 

<div class="alert alert-block alert-info"> 
<b>NOTE</b> The next notebook will focus on another function, `DIVAnd_errormap` to evaluate the error field.
</div>